In [ ]:
#Init Image Transformation
python -m azureml.designer.modules.computer_vision.transform.init_image_transformation.init_image_transformation --output-path DatasetOutputConfig:Output_image_transformation --resize=True --size=256 --center-crop=True --crop-size=224 --pad=False --color-jitter=False --grayscale=False --random-resized-crop=False --random-crop=False --random-horizontal-flip=True --random-vertical-flip=False --random-rotation=False --random-affine=False --random-grayscale=False --random-perspective=False

In [ ]:
#DenseNet
python -m azureml.designer.modules.dl.pytorch.initialize_models.vision.classification.densenet --untrained-model-path DatasetOutputConfig:Untrained_model --model-name=densenet201 --pretrained=True --memory-efficient=False

In [ ]:
#Convert to Image Directory
python -m azureml.designer.modules.computer_vision.preprocess.convert_to_image_directory.convert_to_image_directory --output-path DatasetOutputConfig:Output_image_directory --input-path=$AZUREML_DATAREFERENCE_Input_dataset

In [ ]:
#Split Image Directory 0
python -m azureml.designer.modules.computer_vision.preprocess.split_image_directory.split_image_directory --tgt-train-path DatasetOutputConfig:Output_image_directory1 --tgt-test-path DatasetOutputConfig:Output_image_directory2 --src-path=$AZUREML_DATAREFERENCE_Input_image_directory --fraction=0.9

In [ ]:
#Split Image Directory 1
python -m azureml.designer.modules.computer_vision.preprocess.split_image_directory.split_image_directory --tgt-train-path DatasetOutputConfig:Output_image_directory1 --tgt-test-path DatasetOutputConfig:Output_image_directory2 --src-path=$AZUREML_DATAREFERENCE_Input_image_directory --fraction=0.9

In [ ]:
#Apply Image Transformation 0
python -m azureml.designer.modules.computer_vision.transform.apply_image_transformation.apply_image_transformation --output-path DatasetOutputConfig:Output_image_directory --input-image-transform-path=$AZUREML_DATAREFERENCE_Input_image_transformation --input-image-dir-path=$AZUREML_DATAREFERENCE_Input_image_directory --mode=%22For+training%22

In [ ]:
#Apply Image Transformation 1
python -m azureml.designer.modules.computer_vision.transform.apply_image_transformation.apply_image_transformation --output-path DatasetOutputConfig:Output_image_directory --input-image-transform-path=$AZUREML_DATAREFERENCE_Input_image_transformation --input-image-dir-path=$AZUREML_DATAREFERENCE_Input_image_directory --mode=%22For+inference%22

In [ ]:
#Apply Image Transformation 2
python -m azureml.designer.modules.computer_vision.transform.apply_image_transformation.apply_image_transformation --output-path DatasetOutputConfig:Output_image_directory --input-image-transform-path=$AZUREML_DATAREFERENCE_Input_image_transformation --input-image-dir-path=$AZUREML_DATAREFERENCE_Input_image_directory --mode=%22For+inference%22

In [ ]:
#Train PyTorch Model
python -m azureml.designer.modules.dl.pytorch.train.train_pytorch_model --trained-model-path DatasetOutputConfig:Trained_model --untrained-model-path=$AZUREML_DATAREFERENCE_Untrained_model --train-data-path=$AZUREML_DATAREFERENCE_Training_dataset --valid-data-path=$AZUREML_DATAREFERENCE_Validation_dataset --epochs=2 --batch-size=4 --learning-rate=0.001 --random-seed=1 --patience=3

In [ ]:
#Score Image Model
python -m azureml.designer.modules.dl.pytorch.score.score_image_model.module_invoker --scored-dataset DatasetOutputConfig:Scored_dataset --trained-model=$AZUREML_DATAREFERENCE_Trained_model --dataset=$AZUREML_DATAREFERENCE_Dataset

In [ ]:
#Evaluate Model
python -m azureml.studio.modulehost.module_invoker --module-name=azureml.studio.modules.ml.evaluate.evaluate_generic_module.evaluate_generic_module --evaluation-results DatasetOutputConfig:Evaluation_results --scored-dataset=$AZUREML_DATAREFERENCE_Scored_dataset

In [ ]:
#Look at Confusion Matrix for False Positive Percentages for each class